In [ ]:
from imutils import paths
import os

In [ ]:
import random

In [ ]:
import cv2

In [ ]:
import numpy as np

In [ ]:
# initialize the data and labels
print("[INFO] loading images...")
data = []
labels = []
 
# grab the image paths and randomly shuffle them
imagePaths = sorted(list(paths.list_images("./Data")))
random.seed(42)
random.shuffle(imagePaths)
 
# loop over the input images
for imagePath in imagePaths:
    print(imagePath)
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (64, 64))
    data.append(image)
    label = imagePath.split(os.path.sep)[-2]
    labels.append(label)
 
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

[INFO] loading images...
./Data\normal\276.jpg
./Data\normal\223.jpg
./Data\potholes2\213.jpg
./Data\potholes2\302.jpg
./Data\normal\33.jpg
./Data\potholes2\190.jpg
./Data\potholes\5.jpg
./Data\normal\269.jpg
./Data\normal\194.jpg
./Data\potholes\61.jpg
./Data\normal\50.jpg
./Data\normal\100.jpg
./Data\normal\313.jpg
./Data\potholes2\199.jpg
./Data\normal\229.jpg
./Data\potholes2\319.jpg
./Data\potholes2\215.jpg
./Data\potholes\55.jpg
./Data\normal\205.jpg
./Data\potholes2\258.jpg
./Data\potholes\39.jpg
./Data\normal\103.jpg
./Data\potholes\9.jpg
./Data\potholes\47.jpg
./Data\potholes\32.jpg
./Data\normal\182.jpg
./Data\normal\335.jpg
./Data\normal\134.jpg
./Data\normal\273.jpg
./Data\potholes2\173.jpg
./Data\normal\106.jpg
./Data\normal\177.jpg
./Data\normal\284.jpg
./Data\potholes2\183.jpg
./Data\potholes2\283.jpg
./Data\potholes\48.jpg
./Data\potholes2\176.jpg
./Data\potholes\114.jpg
./Data\potholes2\229.jpg
./Data\potholes2\311.jpg
./Data\potholes\111.jpg
./Data\normal\330.jpg
./Da

./Data\potholes\140.jpg
./Data\normal\69.jpg
./Data\normal\117.jpg
./Data\potholes2\270.jpg
./Data\potholes2\206.jpg
./Data\potholes\27.jpg
./Data\normal\45.jpg
./Data\potholes2\321.jpg
./Data\potholes2\263.jpg
./Data\normal\207.jpg
./Data\normal\94.jpg
./Data\potholes2\304.jpg
./Data\normal\57.jpg
./Data\normal\130.jpg
./Data\potholes\86.jpg
./Data\normal\136.jpg
./Data\normal\334.jpg
./Data\potholes2\236.jpg
./Data\potholes\108.jpg
./Data\potholes\83.jpg
./Data\potholes\79.jpg
./Data\normal\32.jpg
./Data\normal\52.jpg
./Data\potholes2\241.jpg
./Data\potholes2\207.jpg
./Data\normal\244.jpg
./Data\potholes\163.jpg
./Data\potholes2\264.jpg
./Data\normal\35.jpg
./Data\potholes2\191.jpg
./Data\potholes\46.jpg
./Data\normal\125.jpg
./Data\normal\2.jpg
./Data\normal\236.jpg
./Data\potholes\121.jpg
./Data\normal\170.jpg
./Data\normal\324.jpg
./Data\potholes\62.jpg
./Data\normal\88.jpg
./Data\potholes\82.jpg
./Data\potholes2\189.jpg
./Data\potholes2\171.jpg
./Data\potholes2\251.jpg
./Data\pot

In [ ]:
labels[1]
len(labels)

680

In [ ]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)

In [ ]:
lb = LabelBinarizer()
trainY = lb.fit_transform(trainY)
testY = lb.transform(testY)

In [ ]:
testY[0:5]

array([[0, 1, 0],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1]])

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')

(ntrainX, ntestX, ntrainY, ntestY) = train_test_split(data,labels, test_size=0.25, random_state=42)
ntrainY = enc.fit_transform(ntrainY.reshape(-1,1))
ntestY = enc.transform(ntestY.reshape(-1,1))

In [ ]:
import tensorflow
# from keras.preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
aug = ImageDataGenerator(rotation_range=30, width_shift_range=0.1,height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,horizontal_flip=True, fill_mode="nearest")

In [ ]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K

In [ ]:
class SmallVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
            # initialize the model along with the input shape to be
            # "channels last" and the channels dimension itself
            model = Sequential()
            inputShape = (height, width, depth)
            chanDim = -1

            # if we are using "channels first", update the input shape
            # and channels dimension
            if K.image_data_format() == "channels_first":
                inputShape = (depth, height, width)
                chanDim = 1

            # CONV => RELU => POOL layer set
            model.add(Conv2D(32, (3, 3), padding="same",
                input_shape=inputShape))
            model.add(Activation("relu"))
            model.add(BatchNormalization(axis=chanDim))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(0.25))

            # (CONV => RELU) * 2 => POOL layer set
            model.add(Conv2D(64, (3, 3), padding="same"))
            model.add(Activation("relu"))
            model.add(BatchNormalization(axis=chanDim))
            model.add(Conv2D(64, (3, 3), padding="same"))
            model.add(Activation("relu"))
            model.add(BatchNormalization(axis=chanDim))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(0.25))

            # (CONV => RELU) * 3 => POOL layer set
            model.add(Conv2D(128, (3, 3), padding="same"))
            model.add(Activation("relu"))
            model.add(BatchNormalization(axis=chanDim))
            model.add(Conv2D(128, (3, 3), padding="same"))
            model.add(Activation("relu"))
            model.add(BatchNormalization(axis=chanDim))
            model.add(Conv2D(128, (3, 3), padding="same"))
            model.add(Activation("relu"))
            model.add(BatchNormalization(axis=chanDim))
            model.add(MaxPooling2D(pool_size=(2, 2)))
            model.add(Dropout(0.25))

            # first (and only) set of FC => RELU layers
            model.add(Flatten())
            model.add(Dense(512))
            model.add(Activation("relu"))
            model.add(BatchNormalization())
            model.add(Dropout(0.5))

            # softmax classifier
            model.add(Dense(classes))
            model.add(Activation("softmax"))

            # return the constructed network architecture
            return model

In [ ]:
lb.classes_

array(['plain', 'pothole', 'pothole2'], dtype='<U8')

In [ ]:
model = SmallVGGNet.build(width=64, height=64, depth=3,classes=len(lb.classes_))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [ ]:
from keras.optimizers import SGD

In [ ]:
INIT_LR  = 0.01
EPOCHS = 75
BS = 32
 
# initialize the model and optimizer // use 2-class classification)
print("[INFO] training network...")
opt = SGD(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
 
# train the network
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY), steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS)

[INFO] training network...
Instructions for updating:
Use tf.cast instead.
Epoch 1/75
59/59 [==============================] - 7s 123ms/step - loss: 0.8534 - acc: 0.7002 - val_loss: 0.7210 - val_acc: 0.7371
Epoch 2/75
59/59 [==============================] - 2s 40ms/step - loss: 0.6010 - acc: 0.7578 - val_loss: 0.4521 - val_acc: 0.7606
Epoch 3/75
59/59 [==============================] - 2s 41ms/step - loss: 0.5048 - acc: 0.7813 - val_loss: 0.4027 - val_acc: 0.7590
Epoch 4/75
59/59 [==============================] - 2s 41ms/step - loss: 0.4920 - acc: 0.7700 - val_loss: 0.4310 - val_acc: 0.7449
Epoch 5/75
59/59 [==============================] - 2s 42ms/step - loss: 0.4206 - acc: 0.7894 - val_loss: 0.6486 - val_acc: 0.6792
Epoch 6/75
59/59 [==============================] - 3s 45ms/step - loss: 0.4312 - acc: 0.7740 - val_loss: 0.4977 - val_acc: 0.7512
Epoch 7/75
59/59 [==============================] - 3s 45ms/step - loss: 0.4065 - acc: 0.7759 - val_loss: 0.3704 - val_acc: 0.7465
Epoch 8

In [ ]:
import pickle

In [ ]:
model.save("pothole.model")

In [ ]:
f = open("potholelb.pickle", "wb")
f.write(pickle.dumps(lb))
f.close()